<a href="https://colab.research.google.com/github/Akhilesh-K-Yadav/Meta-Learning_Projects/blob/main/Unsupervised/Contrastive_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Contrastive Learning in PyTorch with Point Clouds***

In [1]:
# We will use conda for easier installation of PyG
# If only using pip, it somehow takes forever to install on colab
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [2]:
# Install torch geometric for point-cloud layers
import torch
version = f"https://data.pyg.org/whl/torch-{torch.__version__}.html"
try:
    import torch_geometric
except:
    !echo $version
    !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f $version
    import torch_geometric

In [8]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [9]:
! kaggle datasets download -d peterwarren/shapenet-subset

100% 823M/827M [00:12<00:00, 93.7MB/s]
100% 827M/827M [00:12<00:00, 70.1MB/s]


In [10]:
! unzip /content/shapenet-subset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Output_binvox/87870712d104cf964fc1553be5e987de.binvox  
  inflating: Output_binvox/87bdac1e34f3e6146db2ac45db35c175.binvox  
  inflating: Output_binvox/87c7911287c10ab8407b28046774089c.binvox  
  inflating: Output_binvox/87d536032c36d75a84b9ed79a2c52d47.binvox  
  inflating: Output_binvox/87f103e24f91af8d4343db7d677fae7b.binvox  
  inflating: Output_binvox/881756250fe9a7fa5fceaa39a8353bde.binvox  
  inflating: Output_binvox/8830354625df010037b71c02ebe8cb4d.binvox  
  inflating: Output_binvox/8841a35812d5a95f98a4820926b2a786.binvox  
  inflating: Output_binvox/8843d4bfd7f6cbc19744d3b3ee0cc03.binvox  
  inflating: Output_binvox/884927d82f2de4f435fb6f633afd9a85.binvox  
  inflating: Output_binvox/885a267b2646c762dddd8ce1cda8c6d9.binvox  
  inflating: Output_binvox/88629d904377906e75e93ae4af2fee42.binvox  
  inflating: Output_binvox/8872b51e43d2839b2bbb2dbe46302370.binvox  
  inflating: Output_binvox/88818f1964fc0046830e9cf2cc